In [1]:
# @itsvenu_
# Boltzmann Machines - used in recommender systems

# Boltzmann machine
# Energy based models
# Restricted boltzmann machines
# Contrastive divergence (Gibbs sampling)
# Deep belief networks
# Deep boltzmann machines

# Can we used RBM for patient classification?


In [2]:
!pip install -q torch torchvision

In [2]:
# test installation
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.3340, 0.9728, 0.3398],
        [0.3760, 0.6300, 0.3675],
        [0.4131, 0.4197, 0.5863],
        [0.3932, 0.8460, 0.8345],
        [0.9639, 0.5045, 0.6100]])


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [7]:
movies = pd.read_csv('../../data-dl/ml-1m/movies.dat', 
                      sep = "::", header=None, engine='python',
                     encoding = 'latin-1')
movies.head()
# movie-name # genre

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
users = pd.read_csv('../../data-dl/ml-1m/users.dat', sep = "::", header=None, engine='python', encoding = 'latin-1')
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
ratings = pd.read_csv('../../data-dl/ml-1m/ratings.dat', sep = "::", header=None, engine='python', encoding = 'latin-1')
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
ratings.shape

(1000209, 4)

In [11]:
# ratings
# 0 - user-id
# 1 = movie id
# 2 = rating
# 3 = time-stamp - not really nedded

In [12]:
# base - train set
# test - test set

In [15]:
# user-rating training set
training_set = pd.read_csv('../../data-dl/ml-100k/u1.base', delimiter = '\t', header=None)
training_set.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [16]:
training_set = np.array(training_set, dtype='int')

In [17]:
type(training_set)

numpy.ndarray

In [18]:
test_set = pd.read_csv('../../data-dl/ml-100k/u1.test', delimiter = '\t', header=None)
test_set = np.array(test_set, dtype='int')
test_set[:5]

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198]])

In [21]:
# get number of movies and users
# to prepare a matrix of rows-users; cols-movies; values-ratingsl
# if no rating from a user-i for movie-i, put 0

nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0]))) #max/total number of users
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1]))) # max/total number of movies

print(nb_users)
print(nb_movies)

943
1682


In [26]:
# list of lists - torch compatibility
# in each list, ratings of users

def convert(dat):
    
    new_dat = []
    for id_users in range(1, nb_users+1):
        id_movies = dat[:, 1][dat[:,0] == id_users]
        id_ratings = dat[:, 2][dat[:,0] == id_users]
        i_ratings = np.zeros(nb_movies)
        i_ratings[id_movies-1] = id_ratings
        new_dat.append(list(i_ratings))
    
    return(new_dat)

In [27]:
training_set = convert(training_set)
test_set = convert(test_set)

In [31]:
# make torch-tensors from the training/test dat

training_set = torch.FloatTensor(training_set)
training_set

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [32]:
test_set = torch.FloatTensor(test_set)

In [33]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [34]:
# ratings into binary ratings

training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [35]:
training_set

tensor([[ 1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])

In [54]:
# build RBM
# nodes
# weights
# biases (a and b)

class RBM():
    def __init__(self, nv, nh):
        '''
        nv - number of visible nodes
        nb - number of hidden nodes
        '''
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh) # to make 2d tensor - pytorch compatible
        self.b = torch.randn(1, nv)
        
    def sample_h(self, x):
        '''
        x - visibile neurons, given probability of p(h|v)
        '''
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx) #sigmoid 
        p_h_given_v = torch.sigmoid(activation)
        
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy) #sigmoid 
        p_v_given_h = torch.sigmoid(activation)
        
        return p_v_given_h, torch.bernoulli(p_v_given_h)
        
    # contrastive divergence with gibbs sampling - approximate log-likelihood gradient
    # k - after k-sampling
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
        

In [55]:
nv = len(training_set[0])
nh = 100 # hidden nodes, to identify features
batch_size = 100

In [56]:
rbm = RBM(nv, nh)

In [57]:
rbm

In [58]:
nb_epochs = 10

for epoch in range(1, nb_epochs+1):
    
    train_loss = 0 # loss fun
    s = 0. #counter
    for id_user in range(0, nb_users - batch_size, 100):
        vk = training_set[id_user:id_user + batch_size]
        v0 = training_set[id_user:id_user + batch_size]
        ph0, _ = rbm.sample_h(x=v0)
        
        # k steps of contrastive divergence
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0 < 0] # don't do training on -1 ratings
        
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
        
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))

RuntimeError: The size of tensor a (1682) must match the size of tensor b (100) at non-singleton dimension 1